In [1]:
import json
import pandas as pd

from geojson import Feature
from geojson import FeatureCollection
from geojson import Polygon

from ipyleaflet import Choropleth
from ipyleaflet import Map
from ipywidgets import SelectionRangeSlider

In [2]:
df = pd.read_csv("mesh_timeseries.csv")
unique_dates = list(set(df["record_utc_date"].tolist()))
unique_dates.sort()

In [3]:
def filtering_dataframe(
    df,
    begin_date,
    end_date
):
    df_filtered = df[
        (df["record_utc_date"] >= begin_date) & (df["record_utc_date"] <= end_date)
    ]
    df_filtered = df_filtered.groupby(["id"], as_index=False).agg({
        "record_utc_date": lambda x: begin_date,
        "value": "mean",
        "polygon_geometry": "first",
    })
    return df_filtered

In [4]:
def to_geojson(df: pd.DataFrame) -> tuple[FeatureCollection, dict]:
    features = df[["id", "record_utc_date", "polygon_geometry"]].apply(
        lambda row: Feature(
            data_id=row['id'],
            time=f"{row['record_utc_date']}",
            geometry=Polygon(json.loads(row['polygon_geometry']))
        ), axis=1).tolist()

    feature_collection = FeatureCollection(features=features)
    mapping = {}
    def add_mapping(row):
        mapping[row['id']] = row["value"]

    df[["id", "record_utc_date", "value"]].apply(add_mapping, axis=1).tolist()
    return feature_collection, mapping

def calc_new_data(df: pd.DataFrame, begin_date: str, end_date: str):
    df_filtered = filtering_dataframe(df, begin_date, end_date)
    return to_geojson(df_filtered)

In [5]:
datetime_slider = SelectionRangeSlider(
    description="date: ",
    options=unique_dates,
    index=(0, len(unique_dates)//2),
)

def update_date(change):
    begin_date = min(datetime_slider.value)
    end_date = max(datetime_slider.value)
    feature_collection, mapper = calc_new_data(df, begin_date, end_date)
    try:
        layer.choro_data = mapper
    # note: layer.geo_data may not have some keys at this moment, ignore them.
    #           It will be resolved in the next line(layer.geo_data = ...)
    except KeyError:
        pass
    layer.geo_data = feature_collection
    layer.value_max =  max(mapper.values())
    layer.value_min = min(mapper.values())

datetime_slider.observe(update_date, names='value')
datetime_slider

SelectionRangeSlider(description='date: ', index=(0, 1), options=('2022-02-06T00:00:00Z', '2022-02-06T00:01:00…

In [6]:
feature_collection, mapper = calc_new_data(
    df, min(datetime_slider.value), max(datetime_slider.value))
layer = Choropleth(
    geo_data=feature_collection,
    choro_data=mapper,
    border_color='black',
    key_on='data_id',
    value_min=min(mapper.values()),
    value_max=max(mapper.values()),
    style={
        'fillOpacity': 0.8,
    }
)

# FIXME: should set center/zoom calculated from feature_collection
m = Map(center = (35.667, 139.7), zoom = 7)
m.add_layer(layer)
m

Map(center=[35.667, 139.7], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…